# Manual Cluster Labeling Guide

**Purpose:** Create human-readable labels for each cluster based on their characteristics.

**Process:**
1. Load the clustered data from the previous notebook
2. Review cluster profiles
3. Assign descriptive labels to each cluster
4. Save the labels for use in production

In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

## 1. Load Clustered Data

In [2]:
# Load the data with cluster assignments
df = pd.read_csv('Data_Engineered_with_clusters.csv')
n_clusters = df['cluster'].nunique()

print(f"✅ Loaded {len(df):,} routes with {n_clusters} clusters")
print(f"\n📊 Cluster distribution:")
print(df['cluster'].value_counts().sort_index())

✅ Loaded 24,766 routes with 10 clusters

📊 Cluster distribution:
cluster
0     9490
1      185
2    12440
3        5
4     1560
5      663
6      326
7       28
8       65
9        4
Name: count, dtype: int64


## 2. Review Cluster Characteristics

Use this section to understand what makes each cluster unique.

In [3]:
def profile_cluster(cluster_id):
    """
    Display detailed profile of a cluster to help with labeling.
    """
    cluster_data = df[df['cluster'] == cluster_id]

    print("=" * 100)
    print(f"CLUSTER {cluster_id} - {len(cluster_data):,} routes ({len(cluster_data)/len(df)*100:.1f}%)")
    print("=" * 100)

    # Distance
    dist_mean_km = cluster_data['distance_m'].mean() / 1000
    dist_median_km = cluster_data['distance_m'].median() / 1000
    dist_q25_km = cluster_data['distance_m'].quantile(0.25) / 1000
    dist_q75_km = cluster_data['distance_m'].quantile(0.75) / 1000

    print(f"\n📏 DISTANCE:")
    if dist_median_km < 5:
        distance_desc = "Very Short"
    elif dist_median_km < 15:
        distance_desc = "Short"
    elif dist_median_km < 40:
        distance_desc = "Medium"
    elif dist_median_km < 80:
        distance_desc = "Long"
    else:
        distance_desc = "Very Long"

    print(f"   → {distance_desc} routes")
    print(f"   Mean: {dist_mean_km:.1f} km | Median: {dist_median_km:.1f} km")
    print(f"   Range: Q25={dist_q25_km:.1f} km, Q75={dist_q75_km:.1f} km")

    # Elevation
    ascent_mean = cluster_data['ascent_m'].mean()
    ascent_median = cluster_data['ascent_m'].median()

    print(f"\n⛰️  ELEVATION GAIN:")
    if ascent_median < 50:
        elevation_desc = "Flat"
    elif ascent_median < 200:
        elevation_desc = "Rolling"
    elif ascent_median < 500:
        elevation_desc = "Hilly"
    elif ascent_median < 1000:
        elevation_desc = "Mountainous"
    else:
        elevation_desc = "Alpine"

    print(f"   → {elevation_desc}")
    print(f"   Mean: {ascent_mean:.0f} m | Median: {ascent_median:.0f} m")

    # Surface
    on_road_pct = cluster_data['on_road'].mean()
    off_road_pct = cluster_data['off_road'].mean()
    paved_road_pct = cluster_data['Paved_Road'].mean()
    gravel_pct = cluster_data['Gravel_Tracks'].mean()

    print(f"\n🛣️  SURFACE TYPE:")
    if on_road_pct > 80:
        surface_desc = "Road"
    elif off_road_pct > 60:
        surface_desc = "Off-Road/Gravel"
    else:
        surface_desc = "Mixed"

    print(f"   → {surface_desc}")
    print(f"   On-road: {on_road_pct:.1f}% | Off-road: {off_road_pct:.1f}%")
    print(f"   Paved: {paved_road_pct:.1f}% | Gravel: {gravel_pct:.1f}%")

    # Gradient
    flat_pct = cluster_data['Flat Section'].mean()
    moderate_pct = cluster_data['Moderate Section'].mean()
    steep_pct = cluster_data['Steep Section'].mean()

    print(f"\n📐 GRADIENT PROFILE:")
    if flat_pct > 70:
        gradient_desc = "Mostly Flat"
    elif steep_pct > 30:
        gradient_desc = "Steep Climbs"
    else:
        gradient_desc = "Rolling Hills"

    print(f"   → {gradient_desc}")
    print(f"   Flat: {flat_pct:.1f}% | Moderate: {moderate_pct:.1f}% | Steep: {steep_pct:.1f}%")

    # Turns
    turn_density_mean = cluster_data['Turn_Density'].mean()
    print(f"\n🔄 TURN DENSITY:")
    if turn_density_mean < 0.5:
        turns_desc = "Straight/Few turns"
    elif turn_density_mean < 1.5:
        turns_desc = "Moderate turns"
    else:
        turns_desc = "Winding/Many turns"
    print(f"   → {turns_desc}")
    print(f"   Mean: {turn_density_mean:.2f}")

    # Suggested label
    print(f"\n💡 SUGGESTED LABEL:")
    suggested = f"{distance_desc} {elevation_desc} {surface_desc}"
    print(f"   → '{suggested}'")

    # Sample routes
    print(f"\n📋 SAMPLE ROUTES (random 5):")
    samples = cluster_data.sample(min(5, len(cluster_data)))[['id', 'name', 'distance_m', 'ascent_m', 'region']]
    for idx, route in samples.iterrows():
        print(f"   • {route['name'][:50]:50s} | {route['distance_m']/1000:5.1f}km | {route['ascent_m']:5.0f}m | {route['region']}")

    print("\n")

    return {
        'size': len(cluster_data),
        'distance_desc': distance_desc,
        'elevation_desc': elevation_desc,
        'surface_desc': surface_desc,
        'gradient_desc': gradient_desc,
        'suggested_label': suggested
    }

# Profile all clusters
cluster_insights = {}
for cluster_id in range(n_clusters):
    cluster_insights[cluster_id] = profile_cluster(cluster_id)

CLUSTER 0 - 9,490 routes (38.3%)

📏 DISTANCE:
   → Very Short routes
   Mean: 2.3 km | Median: 0.8 km
   Range: Q25=0.3 km, Q75=2.6 km

⛰️  ELEVATION GAIN:
   → Flat
   Mean: 71 m | Median: 11 m

🛣️  SURFACE TYPE:
   → Mixed
   On-road: 61.9% | Off-road: 1.2%
   Paved: 67.0% | Gravel: 3.5%

📐 GRADIENT PROFILE:
   → Rolling Hills
   Flat: 3.3% | Moderate: 41.9% | Steep: 3.7%

🔄 TURN DENSITY:
   → Winding/Many turns
   Mean: 1.53

💡 SUGGESTED LABEL:
   → 'Very Short Flat Mixed'

📋 SAMPLE ROUTES (random 5):
   • Unnamed route                                      |   1.1km |    20m | eastcoastus
   • Unnamed route                                      |   0.8km |    19m | tile_ita_west
   • Unnamed route                                      |   4.1km |    19m | belgium
   • Unnamed route                                      |   0.0km |     0m | netherlands
   • Lanark (Kirkfieldbank) to Biggar                   |   8.7km |   213m | UK2_Data


CLUSTER 1 - 185 routes (0.7%)

📏 DISTANCE:
   → 

## 3. Create Cluster Labels

Based on the profiles above, assign descriptive labels to each cluster.

**Labeling Tips:**
- Keep labels concise (2-4 words)
- Focus on the most distinctive characteristics
- Use cyclist-friendly language (e.g., "gravel adventure" vs "off-road")
- Think about what would help users understand the curveball recommendation

**Example good labels:**
- "Short flat city ride"
- "Mountain climbing challenge"
- "Gravel adventure"
- "Long road tour"
- "Quick urban spin"
- "Rolling countryside"

In [6]:
# MANUAL LABELING: Edit this dictionary based on cluster profiles above
CLUSTER_LABELS = {
    0: "Short and flat",  # Replace with actual label after reviewing profiles
    1: "Very long alpine mixed",
    2: "Very short and flat",
    3: "Mixed surface multi-day tours",
    4: "Mid-length rolling hills",
    5: "Very Short Flat Mixed",
    6: "Long alpine mixed",
    7: "Very long rolling hills",
    8: "Very short flat mixed",
    9: "Exceptionally long alpine steep",
    # Add more if you have more clusters
}

# Trim to actual number of clusters
CLUSTER_LABELS = {k: v for k, v in CLUSTER_LABELS.items() if k < n_clusters}

print("📝 Current labels:")
for cluster_id, label in CLUSTER_LABELS.items():
    print(f"   Cluster {cluster_id}: '{label}'")

📝 Current labels:
   Cluster 0: 'Short and flat'
   Cluster 1: 'Very long alpine mixed'
   Cluster 2: 'Very short and flat'
   Cluster 3: 'Mixed surface multi-day tours'
   Cluster 4: 'Mid-length rolling hills'
   Cluster 5: 'Very Short Flat Mixed'
   Cluster 6: 'Long alpine mixed'
   Cluster 7: 'Very long rolling hills'
   Cluster 8: 'Very short flat mixed'
   Cluster 9: 'Exceptionally long alpine steep'


## 4. Review Labels with Cluster Stats

In [7]:
# Create summary table
summary_data = []

for cluster_id in range(n_clusters):
    cluster_data = df[df['cluster'] == cluster_id]

    summary_data.append({
        'Cluster': cluster_id,
        'Label': CLUSTER_LABELS.get(cluster_id, 'NOT LABELED'),
        'Size': len(cluster_data),
        'Avg Distance (km)': f"{cluster_data['distance_m'].mean()/1000:.1f}",
        'Avg Ascent (m)': f"{cluster_data['ascent_m'].mean():.0f}",
        'On-Road %': f"{cluster_data['on_road'].mean():.0f}",
        'Flat %': f"{cluster_data['Flat Section'].mean():.0f}"
    })

summary_df = pd.DataFrame(summary_data)

print("=" * 120)
print("CLUSTER LABELS SUMMARY")
print("=" * 120)
print(summary_df.to_string(index=False))
print("=" * 120)

CLUSTER LABELS SUMMARY
 Cluster                           Label  Size Avg Distance (km) Avg Ascent (m) On-Road % Flat %
       0                  Short and flat  9490               2.3             71        62      3
       1          Very long alpine mixed   185             128.0           1392        63     71
       2             Very short and flat 12440               3.0             22        65     97
       3   Mixed surface multi-day tours     5             645.9           6499        65     79
       4        Mid-length rolling hills  1560              33.0            417        61     66
       5           Very Short Flat Mixed   663               0.2              2        67     60
       6               Long alpine mixed   326              56.7           1704        60     18
       7         Very long rolling hills    28             354.3           5108        64     54
       8           Very short flat mixed    65               0.0              0        64     97
       

## 5. Save Labels for Production Use

In [8]:
# Save as JSON
with open('cluster_labels.json', 'w') as f:
    json.dump(CLUSTER_LABELS, f, indent=2)

print("✅ Labels saved as 'cluster_labels.json'")

# Save as Python file for easy import
with open('cluster_labels.py', 'w') as f:
    f.write('"""\n')
    f.write('Cluster labels for route recommendations.\n')
    f.write('\n')
    f.write('Generated by: 2).Manual_Cluster_Labeling.ipynb\n')
    f.write('"""\n\n')
    f.write('CLUSTER_LABELS = {\n')
    for cluster_id, label in sorted(CLUSTER_LABELS.items()):
        f.write(f'    {cluster_id}: "{label}",\n')
    f.write('}\n')

print("✅ Labels saved as 'cluster_labels.py'")
print("\n📦 To use in production, copy 'cluster_labels.py' to 'cycle_more/api_faf/app/'")

✅ Labels saved as 'cluster_labels.json'
✅ Labels saved as 'cluster_labels.py'

📦 To use in production, copy 'cluster_labels.py' to 'cycle_more/api_faf/app/'


## 6. Test Label Descriptions

In [9]:
# Test: Show random routes with their cluster labels
print("🧪 TESTING: Random routes with cluster labels\n")

sample_routes = df.sample(10)
for idx, route in sample_routes.iterrows():
    cluster_id = route['cluster']
    label = CLUSTER_LABELS.get(cluster_id, 'UNKNOWN')

    print(f"Route: {route['name'][:50]:50s}")
    print(f"  → Cluster {cluster_id}: '{label}'")
    print(f"  → Stats: {route['distance_m']/1000:.1f}km, {route['ascent_m']:.0f}m ascent, {route['on_road']:.0f}% on-road")
    print()

🧪 TESTING: Random routes with cluster labels

Route: Unnamed route                                     
  → Cluster 2: 'Very short and flat'
  → Stats: 1.2km, 8m ascent, 67% on-road

Route: Bader Way Cycleway                                
  → Cluster 2: 'Very short and flat'
  → Stats: 1.4km, 12m ascent, 100% on-road

Route: Unnamed route                                     
  → Cluster 2: 'Very short and flat'
  → Stats: 12.8km, 82m ascent, 84% on-road

Route: Unnamed route                                     
  → Cluster 2: 'Very short and flat'
  → Stats: 1.8km, 9m ascent, 100% on-road

Route: Amper Glonn                                       
  → Cluster 0: 'Short and flat'
  → Stats: 0.0km, 0m ascent, 100% on-road

Route: Unnamed route                                     
  → Cluster 0: 'Short and flat'
  → Stats: 0.9km, 5m ascent, 0% on-road

Route: Unnamed route                                     
  → Cluster 0: 'Short and flat'
  → Stats: 0.1km, 0m ascent, 0% on-road

Route:

## Summary

**Next Steps:**
1. Review the cluster profiles in section 2
2. Edit the `CLUSTER_LABELS` dictionary in section 3 with meaningful names
3. Run sections 4-6 to save and verify your labels
4. Copy `cluster_labels.py` to `cycle_more/api_faf/app/` for production use

**Production Integration:**
```python
from cluster_labels import CLUSTER_LABELS

# When serving curveball recommendations:
user_cluster = kmeans.predict(user_features_scaled)[0]
user_cluster_label = CLUSTER_LABELS[user_cluster]

# Get curveball from different cluster
different_clusters = [c for c in range(n_clusters) if c != user_cluster]
curveball_cluster = random.choice(different_clusters)
curveball_label = CLUSTER_LABELS[curveball_cluster]

# Display to user:
# "Your route is a '{user_cluster_label}'. Try this '{curveball_label}' for something different!"
```